# Binary Logistic Regression From Scratch 

* Using the first 2 classes of the iris dataset.
* Compared to the sklearn counterpart at the end of the notebook.

In [23]:
import sys, os
import numpy as np
import pandas as pd
import sklearn.datasets as dataset
from sklearn.model_selection import train_test_split
dir_path = os.getcwd()
sys.path.insert(0, dir_path + '/../data_tools')
from sklearn_pandas_converter import sklearn_to_df

## Load and Format Data

In [28]:
skdata = dataset.load_iris()
# Convert sklearn dataset to pandas dataframe
data = sklearn_to_df(skdata)
# Use only first two classses for 2 class logistic regression
data = data[data['target'] != 2]
data.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [29]:
# Split data into features and classes
x = data.drop(['target'], axis = 1)
y = data['target']
# Split data into train and test data
x_train, x_test, y_train, y_test = train_test_split(x,y,train_size = 0.85, test_size = 0.15, random_state = 0)
x_train = np.array(x_train)
y_train = np.array(y_train)

## Build the Model

In [12]:
class logistic_regression:

    def __init__(self):
        self.weights = []

    def sigmoid(self, x):
        z = np.dot(x, self.weights)
        return 1 / (1 + np.exp(-z))

    def gradient_descent(self, x, y, y_hat):
        return np.dot(x.T, (y_hat - y)) / y.shape[0]

    def update_weights(self, rate, gradient):
        return self.weights - rate * gradient

    def train(self, features, targets, steps=1000, rate=0.1, verbose=False):
        intercepts = np.ones((features.shape[0], 1))
        features = np.concatenate((intercepts, features), axis=1)
        self.weights = np.ones(features.shape[1])

        for i in range(steps + 1):
            y_hat = self.sigmoid(features)
            gradient = self.gradient_descent(features, targets, y_hat)
            self.weights = self.update_weights(rate, gradient)

            if i % 100 == 0 and verbose == True:
                print(f'Step: {i}. Gradient: {gradient}. Weights: {self.weights}\n')

        return self.weights

    def predict(self, test_features):
        test_data = np.concatenate((np.ones((test_features.shape[0], 1)), test_features), axis = 1)
        predictions = np.round(self.sigmoid(test_data))
        predictions = [int(i) for i in predictions]
        return predictions

## Using the Model

### Training the Model
* Calculating the weights associated with the data
* Hyperparameters are:
  - number of steps (often called epochs)
  - Learning rate or the step size

In [13]:
# Create a instance of the logistic regression class
clf = logistic_regression()

In [14]:
# Calculate weights with the train method in the class
weights = clf.train(x_train, y_train, 500, 0.1, verbose=True)

Step: 0. Gradient: [0.52940164 2.65759832 1.81761505 0.7799851  0.12940926]. Weights: [0.94705984 0.73424017 0.81823849 0.92200149 0.98705907]

Step: 100. Gradient: [ 0.00661408  0.00944919  0.03771522 -0.05550382 -0.02407779]. Weights: [ 0.57229281 -0.69856723 -0.76393502  1.61995933  1.38187996]

Step: 200. Gradient: [ 0.00354531  0.00503312  0.02014556 -0.02979185 -0.01310265]. Weights: [ 0.52479152 -0.76618059 -1.03423008  2.01871217  1.55603137]

Step: 300. Gradient: [ 0.00242248  0.00343499  0.01376034 -0.02041241 -0.00906403]. Weights: [ 0.49572047 -0.8074202  -1.19937448  2.26331295  1.66412365]

Step: 400. Gradient: [ 0.00184124  0.00261148  0.01046284 -0.01555706 -0.00695977]. Weights: [ 0.47469766 -0.83723138 -1.31880853  2.44069172  1.74318269]

Step: 500. Gradient: [ 0.00148594  0.00210928  0.00844892 -0.01258648 -0.00566551]. Weights: [ 0.45820729 -0.86062876 -1.41254176  2.58019547  1.80578529]



### Predicting with the Model

In [15]:
predictions = clf.predict(x_test)
predictions

[0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0]

## Sklearn Logistic Regression

In [16]:
from sklearn.linear_model import LogisticRegression
skclf = LogisticRegression()

In [17]:
# Fit model to data
skclf.fit(x_train, y_train)

C:\Users\wardz\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [18]:
# Make prediction
sk_predictions = skclf.predict(x_test)
sk_predictions

array([0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0])

## Comparing the Models

In [19]:
outcome = pd.DataFrame(
    {'Scratch': predictions,
     'Sklearn': sk_predictions,
     'Actual': y_test
     })

In [20]:
# Values scratch classifier missed
outcome[outcome['Scratch'] != outcome['Actual']]

,Scratch,Sklearn,Actual


In [21]:
# Values sklearn classifier missed
outcome[outcome['Sklearn'] != outcome['Actual']]

,Scratch,Sklearn,Actual


Both the built with numpy and pandas logistic regression model and the sklearn equivelent correctly predicted all of the test examples in the dataset. 